<a href="https://colab.research.google.com/github/YasinnSoylu/montecarlo_similasyon_getiri_olas-l-k/blob/main/montecarlo_similasyon_getiri_olas%C4%B1l%C4%B1k__(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Microsoft Hisse Senedi Fiyat Simülasyonu ve Risk Analizi"

In [68]:
# Import the necessary libraries
import yfinance as yf
import numpy as np
import pandas as pd
from IPython.display import clear_output


# Create a Ticker object for Microsoft (MSFT)
msft = yf.Ticker("MSFT")

# Get the current price of MSFT
current_price = msft.history(period='1y')['Close'][0]
current_price

257.0948181152344

In [69]:
# Set the number of simulations and the number of days to simulate
num_simulations = 1000
num_days = 20

In [70]:
# Calculate the daily returns based on the past 3 months of MSFT's historical prices
daily_returns = np.log(1 + msft.history(period='3mo').Close.pct_change())

# Calculate the mean and standard deviation of the daily returns
mu = daily_returns.mean()
sigma = daily_returns.std()

In [71]:
# Simulate the price of MSFT for the next num_days days using a geometric Brownian motion model
# Store the results in a DataFrame called simulated_price_series
simulated_price_series = pd.DataFrame()
for i in range(num_simulations):
  simulated_price_series[i] = [current_price * np.exp(np.random.normal(mu, sigma)) for j in range(num_days)];
clear_output()


In [72]:
# Calculate the minimum, maximum, and average simulated prices at the end of the simulation period
minimum_price = simulated_price_series.min(axis=1)[num_days-1]
maximum_price = simulated_price_series.max(axis=1)[num_days-1]
average_price = simulated_price_series.mean(axis=1)[num_days-1]

In [73]:
# Calculate the returns of each simulation and the probability of a loss
returns = (simulated_price_series - current_price) / current_price
negative_returns = returns[returns < 0]
probability_of_loss = len(negative_returns) / num_simulations

In [74]:
print(f"Current price of company stock: {current_price:.2f} dollars")
print(f"Forecast stock prices for the next 3 months:")
print(f"Minimum estimate: {minimum_price:.2f} dollars")
print(f"Average estimate: {average_price:.2f} dollars")
print(f"Maximum estimate: {maximum_price:.2f} dollars")

Current price of company stock: 257.09 dollars
Forecast stock prices for the next 3 months:
Minimum estimate: 243.76 dollars
Average estimate: 258.08 dollars
Maximum estimate: 272.65 dollars


In [75]:
if probability_of_loss > 0.5:
     print("High probability of loss in 3 months.")
else:
     print("High probability of making a profit in 3 months.")

High probability of making a profit in 3 months.


# portföy  Fiyat Simülasyonu ve Risk Analizi

In [76]:
# Investment amount
investment_amount = 10000

# Ticker symbols for stocks in the portfolio
tickers = ["MSFT", "AAPL","TSLA","GS"]

close_prices = yf.download(tickers, start='2022-04-01', end='2023-04-01',interval="1d").fillna(method='ffill')['Adj Close']

[*********************100%***********************]  4 of 4 completed


In [77]:
# Define the weights of the portfolio
weights = [0.25, 0.25,0.25,0.25]

# Calculate the current value of the portfolio
# We use a DataFrame containing the latest prices
current_value = np.dot(close_prices.iloc[-1], weights)

In [78]:
# Perform Monte Carlo simulation for the next 3 months
num_simulations = 1000
num_days = 90

In [79]:
# Calculate daily returns
daily_returns = np.log(1 + close_prices.pct_change())

# Calculate mean and standard deviation of the daily returns
mu = daily_returns.mean()
sigma = daily_returns.std()

In [80]:
# Create an empty DataFrame to store the simulated prices
simulated_price_series = pd.DataFrame()

# Loop over each simulation
for i in range(num_simulations):
    # Generate a random sample of daily returns
    daily_returns_sample = np.random.normal(mu, sigma, size=(num_days, len(tickers)))

    # Initialize the price matrix with the latest prices
    price_matrix = np.zeros((num_days, len(tickers)))
    price_matrix[0] = close_prices.iloc[-1]

    # Simulate the price series for the next 3 months
    for j in range(1, num_days):
        price_matrix[j] = price_matrix[j-1] * np.exp(daily_returns_sample[j])

    # Calculate the simulated price for each simulation
    simulated_price_series[i] = np.dot(price_matrix, weights)
    clear_output()

In [81]:
# Calculate the estimated portfolio price for the next 3 months
minimum_price = simulated_price_series.min()[num_days-1]
maximum_price = simulated_price_series.max()[num_days-1]
average_price = simulated_price_series.mean()[num_days-1]

In [82]:
# Let's print current and estimated prices
print(f"Current value of portfolio: {current_value:.2f} dollars")
print(f"Estimated portfolio values for the next 3 months:")
print(f"Minimum estimate: {minimum_price:.2f} dollars")
print(f"Average estimate: {average_price:.2f} dollars")
print(f"Max estimate: {minimum_price:.2f} dollars")

Current value of portfolio: 246.11 dollars
Estimated portfolio values for the next 3 months:
Minimum estimate: 237.63 dollars
Average estimate: 271.84 dollars
Max estimate: 237.63 dollars


In [83]:
# Calculate the returns, lower and upper quantiles, and the risk of loss of a portfolio
# by simulating its prices using Monte Carlo simulations

returns = (simulated_price_series - current_value) / current_value
lower_quantile = returns.quantile(0.05)
upper_quantile = returns.quantile(0.95)
risk_of_loss = (returns < 0).mean()


In [84]:
print(f"\nThe current value of the portfolio: {current_value:.2f} dollars")
print(f"Profit/loss for the next 3 months:")
print(f"Maximum loss with 5% probability: {lower_quantile.iloc[0]:.2%}")
print(f"Maximum gain with 5% probability: {upper_quantile.iloc[0]:.2%}")
print(f"Probability of damage: {risk_of_loss.iloc[0]:.2%}")
print(f"Probability of damage: {1-risk_of_loss.iloc[0]:.2%}")



The current value of the portfolio: 246.11 dollars
Profit/loss for the next 3 months:
Maximum loss with 5% probability: -5.59%
Maximum gain with 5% probability: 1.18%
Probability of damage: 86.67%
Probability of damage: 13.33%


In [85]:
returns = (simulated_price_series - current_value) / current_value
lower_quantile = returns.quantile(0.10)
upper_quantile = returns.quantile(0.90)
risk_of_loss = (returns < 0).mean()

In [86]:
print(f"\nThe current value of the portfolio: {current_value:.2f} dollars")
print(f"Profit/loss for the next 3 months:")
print(f"Maximum loss with 10% probability: {lower_quantile.iloc[0]:.2%}")
print(f"Maximum gain with 10% probability: {upper_quantile.iloc[0]:.2%}")
print(f"Probability of damage: {risk_of_loss.iloc[0]:.2%}")
print(f"Probability of damage: {1-risk_of_loss.iloc[0]:.2%}")



The current value of the portfolio: 246.11 dollars
Profit/loss for the next 3 months:
Maximum loss with 10% probability: -4.73%
Maximum gain with 10% probability: 0.31%
Probability of damage: 86.67%
Probability of damage: 13.33%


In [87]:
returns = (simulated_price_series - current_value) / current_value
lower_quantile = returns.quantile(0.15)
upper_quantile = returns.quantile(0.85)
risk_of_loss = (returns < 0).mean()

In [88]:
print(f"\nThe current value of the portfolio: {current_value:.2f} dollars")
print(f"Profit/loss for the next 3 months:")
print(f"Maximum loss with 15% probability: {lower_quantile.iloc[0]:.2%}")
print(f"Maximum gain with 15% probability: {upper_quantile.iloc[0]:.2%}")
print(f"Probability of damage: {risk_of_loss.iloc[0]:.2%}")
print(f"Probability of damage: {1-risk_of_loss.iloc[0]:.2%}")


The current value of the portfolio: 246.11 dollars
Profit/loss for the next 3 months:
Maximum loss with 15% probability: -4.23%
Maximum gain with 15% probability: -0.31%
Probability of damage: 86.67%
Probability of damage: 13.33%
